# HAVENT DONE AT AND ATAE MODELS 
# ALSO 3 CHANNEL AND 5 CHANNEL CNN

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import math
import pickle
from sklearn.model_selection import KFold
import json

In [2]:
from keras.callbacks import EarlyStopping
from Attention import Attention
from keras.models import load_model
from keras.layers import Input, Embedding, LSTM,Bidirectional,Dense
from keras.models import Model,Sequential
from keras.optimizers import Adam
from keras import backend as K
from keras.models import load_model
from sklearn.metrics import confusion_matrix,f1_score,precision_score,recall_score,accuracy_score



Using TensorFlow backend.


In [3]:
import skopt
from skopt import gp_minimize, forest_minimize
from skopt.space import Real, Categorical, Integer
from skopt.plots import plot_convergence
from skopt.plots import plot_objective, plot_evaluations
from skopt.utils import use_named_args

In [4]:
#from skopt.plots import plot_histogram, plot_objective_2D

In [5]:
import keras

In [6]:
tf.__version__

'1.8.0'

In [7]:
keras.__version__

'2.2.0'

In [8]:
skopt.__version__

'0.5.2'

In [9]:
dim_learning_rate = Real(low=1e-6, high=1e-2, prior='log-uniform',
                         name='learning_rate')

In [10]:
dropout = Real(low=0.2, high=0.9,
                         name='dropout')

In [11]:
dim_num_hidden_layers = Integer(low=1, high=5, name='num_hidden_layers')


In [12]:
para_batch_size = Categorical(categories=[8,16,32,64],name='batch_size')


In [13]:
dim_num_dense_nodes = Integer(low=5, high=512, name='num_dense_nodes')


In [14]:
lstm_out = Integer(low=100, high=400, name='lstm_out')

In [15]:
dim_activation = Categorical(categories=['relu', 'sigmoid','tanh'],name='activation')

In [16]:
parameters = [dim_learning_rate,
              dim_num_hidden_layers,
              dim_num_dense_nodes,
              dim_activation,
              dropout,
              para_batch_size,
              lstm_out]

In [17]:
default_parameters = [0.0006973763486468701, 1, 300, 'relu',0.2,8,300]


In [18]:
a=pickle.load(open("D:/PythonCodes/Sentiment-Analysis/Data/all_data.dat","rb"))


In [19]:
a.keys()

dict_keys(['train_sentiment', 'h_aspect_encoding', 'sentence', 'p_sentiment', 'lable_encoding', 'h_target', 'p_aspect', 'p_aspect_encoding', 'vocab_size', 'embedding_matrix', 'aspect', 'p_target', 'target', 'h_sentence', 'h_sentiment', 'h_aspect', 'input_aspect', 'p_sentence'])

In [20]:
dataX= a['sentence']
aspect=a['aspect']
test_headline_X=a['h_sentence']
test_post_X=a['p_sentence']
h_aspect=a['h_aspect']
p_aspect=a['p_aspect']
label_encoding=a['lable_encoding']
h_target=a['h_target']
target=a['target']


In [21]:
def get_class_from_pred(pred):
    return [label_encoding[x.argmax()] for x in pred]

In [22]:
aspect_encoded = get_class_from_pred(aspect)
h_aspect_encoding=a['h_aspect_encoding']
p_aspect_encoding=a['p_aspect_encoding']

In [23]:
max_length=11
vocab_size=a['vocab_size']
embedding_matrix=a['embedding_matrix']

# Our Model+Attention

In [24]:
def define_model(learning_rate,dropout,lstm_out,num_hidden_layers,num_dense_nodes,em,em_trainable_flag,em_dim,activation):
     
#   model=Sequential()
#    model.add(Embedding(vocab_size, len(eval(em)[0]), weights = [eval(em)],input_length=max_length,trainable = em_trainable_flag))
#    model.add(Bidirectional(LSTM(lstm_out, return_sequences=True,recurrent_dropout=dropout)))
#    model.add(Attention())
    
#    for i in range(num_hidden_layers):
#        model.add(Dense(num_dense_nodes,activation=activation))
        
#    model.add(Dense(27,activation='softmax'))


    input_context= Input(shape=(max_length,),name='Context')
    
    embedding=Embedding(vocab_size, len(eval(em)[0]), weights = [eval(em)],input_length=max_length,trainable = em_trainable_flag)
    context= embedding(input_context)
        
    a = Bidirectional(LSTM(300, return_sequences=True,recurrent_dropout=dropout))(context)
    
    #print(a.shape)

    alpha = Attention()(a)
    
    x=Dense(int((2*lstm_out+27)/2),activation='relu')(alpha)
        
    out=Dense(27,activation='softmax')(x)
    
    model= Model(inputs=input_context ,outputs=out) 
    
    optimizer = Adam(lr=learning_rate)
    
    model.compile(loss = 'categorical_crossentropy', optimizer=optimizer,metrics = ['accuracy'])
    
    return model

In [25]:
path_best_model = 'our_model_attention.keras'


In [26]:
record = dict()
key=0

In [27]:
@use_named_args(dimensions=parameters)

def fitness(learning_rate,num_hidden_layers,num_dense_nodes,activation,dropout,batch_size,lstm_out):
        global key
        global record

        # Print the hyper-parameters.
        print('-----------------------------combination no={0}------------------'.format(key))
        print('learning rate===> '.format(learning_rate))
        print('num_hidden_layers:', num_hidden_layers)
        print('num_dense_nodes:', num_dense_nodes)
        print('activation:', activation)
        print('lstm_out:', lstm_out)
        print('dropout===>',dropout)
        print('batch_size===> ',batch_size)

        print()



        acc=[]
        f1_head=[]
        f1_post=[]
        
        #for train,test in kfold.split(dataX,aspect):
        model = define_model(learning_rate=learning_rate,
                             num_hidden_layers=num_hidden_layers,
                             num_dense_nodes=num_dense_nodes,
                             activation=activation,
                             lstm_out=lstm_out,
                             dropout=dropout,
                             em='embedding_matrix',
                             em_trainable_flag=False,
                             em_dim=300)
        
        #graph = tf.get_default_graph()
    

        history = model.fit(x=dataX,
                            y=aspect,
                            epochs=1,
                            batch_size=batch_size
                            )

      


        accuracy = history.history['acc'][-1]

        # Print the classification accuracy.
        print()
        print("Accuracy: {0:.2%}".format(accuracy))
        print()
        
        
        _,accuracy_1= model.evaluate(x=test_headline_X , y=h_aspect_encoding)

        pred1 = model.predict(test_headline_X)
        pred_class = get_class_from_pred(pred1)
        f1_1=f1_score(h_aspect,pred_class,average='macro')

        _,accuracy_2 = model.evaluate(x=test_post_X , y=p_aspect_encoding)


        pred2= model.predict(test_post_X)
        pred_class = get_class_from_pred(pred2)
        f1_2=f1_score(p_aspect,pred_class,average='macro')

        print("Test Headline")
        print("F1 Score: ",f1_1)
        print("Accuracy: ",accuracy_1)
        print()

        print("Test Post")
        print("F1 Score: ",f1_2)
        print("Accuracy: ",accuracy_2)
        print()


        f1_head.append(f1_1)
        f1_post.append(f1_2)

        ###########################

        record[key] = {'parameters':[learning_rate,num_hidden_layers,num_dense_nodes,activation,dropout,batch_size,lstm_out],
               'accuracy_Train':accuracy,
               'f1_score_headlines':f1_1,
               'f1_score_post':f1_2,
               'accuracy_Headline_Test':accuracy_1,
               'accuracy_Post_Test':accuracy_2,

              }

        pickle.dump(record, open(r"D:\PythonCodes\Sentiment-Analysis\Data\record_parameters.dat","wb"))

        
        key+=1

        del model
        
        print("model deleted")

        K.clear_session()
        
        print("Session cleared ")
        
        return -accuracy




In [28]:
seed=1
np.random.seed(seed)

In [29]:
#kfold = KFold(n_splits=10, shuffle=True, random_state=seed)

In [30]:

#fitness(x=default_parameters)


In [ ]:
search_result = gp_minimize(func=fitness,
                            dimensions=parameters,
                            acq_func='EI', # Expected Improvement.
                            n_calls=11,
                            x0=default_parameters)

-----------------------------combination no=0------------------
learning rate===> 
num_hidden_layers: 1
num_dense_nodes: 300
activation: relu
lstm_out: 300
dropout===> 0.2
batch_size===>  8

Epoch 1/1
1173/1173 [==============================] - 23s 20ms/step - loss: 2.1885 - acc: 0.4339

Accuracy: 43.39%

64/99 [==================>...........] - ETA: 0s

C:\Users\simcy\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


99/99 [==============================] - 0s 2ms/step


C:\Users\simcy\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


Test Headline
F1 Score:  0.06690439139865055
Accuracy:  0.03225806447607215

Test Post
F1 Score:  0.18065268065268067
Accuracy:  0.0

model deleted
Session cleared 
-----------------------------combination no=1------------------
learning rate===> 
num_hidden_layers: 3
num_dense_nodes: 134
activation: tanh
lstm_out: 389
dropout===> 0.6150142450871858
batch_size===>  16



In [ ]:
space = search_result.space


In [ ]:
space.point_to_dict(search_result.x)


In [ ]:
search_result.fun


In [ ]:
print("Training Done")

# CNN+LSTM_ATTENTION

para_filter_size = Integer(low=1,high=6,name = 'filter_size')
para_n_filters = Categorical(categories=[100,200,300,400],name='n_filters')

default_parameters_2=[0.0006973763486468701, 1, 300, 'relu', 0.2, 8, 300,3,100]

parameters_2=[dim_learning_rate,
              dim_num_hidden_layers,
              dim_num_dense_nodes,
              dim_activation,
              dropout,
              para_batch_size,
              lstm_out,
              para_filter_size,
              para_n_filters]

def define_model_1(learning_rate,dropout,lstm_out,num_hidden_layers,num_dense_nodes,em,em_trainable_flag,em_dim,activation,n_filters,kernel_size):
     
    model=Sequential()
    model.add(Embedding(vocab_size, len(eval(em)[0]), weights = [eval(em)],input_length=max_length,trainable = em_trainable_flag))
    model.add(Dropout(dropout))
    model.add(Conv1D(n_filters,kernel_size=3,activation=activation))
    model.add(LSTM(lstm_out, return_sequences=True,recurrent_dropout=dropout))
    model.add(Attention())
    
    for i in range(num_hidden_layers):
        model.add(Dense(num_dense_nodes,activation=activation))
        
    model.add(Dense(27,activation='softmax'))
        
    optimizer = Adam(lr=learning_rate)
    
    model.compile(loss = 'categorical_crossentropy', optimizer=optimizer,metrics = ['accuracy'])
    
    return model

path_best_model = 'cnn_lstm_attention.keras'


record = dict()
key=0

def get_class_from_pred(pred):
    return [label_encoding[x.argmax()] for x in pred]

@use_named_args(dimensions=parameters_2)
def fitness(learning_rate,num_hidden_layers,num_dense_nodes,activation,dropout,batch_size,epoch,lstm_out,filter_size,n_filters):
    global key
    global record
    
    # Print the hyper-parameters.
    print('-----------------------------combination no={0}------------------'.format(key))
    print('learning rate===> '.format(learning_rate))
    print('num_hidden_layers:', num_hidden_layers)
    print('num_dense_nodes:', num_dense_nodes)
    print('activation:', activation)
    print('lstm_out:', lstm_out)
    print('dropout===>'.format(dropout))
    print('batch_size===> ',batch_size)
    print('filter_size===> ',filter_size)
    print('num_filters===> 'n_filters)

    print()
    
    
    acc=[]
    f1_Score=[]
    for train,test in kfold.split(dataX,aspect):
        model = define_model_1(learning_rate=learning_rate,
                             num_hidden_layers=num_hidden_layers,
                             num_dense_nodes=num_dense_nodes,
                             activation=activation,
                             lstm_out=lstm_out,
                             dropout=dropout,
                             em='embedding_matrix',
                             em_trainable_flag=False,
                             em_dim=300,
                             n_filters=n_filters,
                             kernel_size=filter_size)

  
   
        history = model.fit(x=dataX,
                            y=aspect,
                            epochs=10,
                            batch_size=batch_size,
                            )

        pred1 = model.predict(test_headline_X)
        pred_class = get_class_from_pred(pred1)
        f1=f1_score(h_aspect,pred_class,average='macro')
        
        accuracy = history.history['acc'][-1]

        # Print the classification accuracy.
        print()
        print("Accuracy: {0:.2%}".format(accuracy))
        print("F1 Score: ",f1)
        print()
        
        f1_Score.append(f1)
        acc.append(accuracy)
        
        record[key] = {'parameters':[learning_rate,num_hidden_layers,num_dense_nodes,activation,dropout,batch_size,lstm_out,filter_size,n_filters],'f1_score':f1,'accuracy':accuracy}
        
        with open('record.json', 'w') as fout:
               json.dump(record,fout,indent=4)
    
        key+=1
    
        del model

        K.clear_session()
        return -accuracy

        
        

seed=7
np.random.seed(seed)

kfold = KFold(n_splits=10, shuffle=True, random_state=seed)

fitness(x=default_parameters_2)


search_result = gp_minimize(func=fitness,
                            dimensions=parameters_2,
                            acq_func='EI', # Expected Improvement.
                            n_calls=11,
                            x0=default_parameters_2)

space = search_result.space


space.point_to_dict(search_result.x)


search_result.fun


sorted(zip(search_result.func_vals, search_result.x_iters))


# CNN+ Bi Directional LSTM_ATTENTION

para_filter_size = Integer(low=1,high=6,name = 'filter_size')
para_n_filters = Categorical(categories=[100,200,300,400],name='n_filters')

default_parameters_2=[0.0006973763486468701, 1, 300, 'relu', 0.2, 8, 10, 300,3,100]

parameters_2=[dim_learning_rate,
              dim_num_hidden_layers,
              dim_num_dense_nodes,
              dim_activation,
              dropout,
              para_batch_size,
              para_epoch,
              lstm_out,
              para_filter_size,
              para_n_filters]

def define_model_2(learning_rate,dropout,lstm_out,num_hidden_layers,num_dense_nodes,em,em_trainable_flag,em_dim,activation,n_filters,kernel_size):
     
    model=Sequential()
    model.add(Embedding(vocab_size, len(eval(em)[0]), weights = [eval(em)],input_length=max_length,trainable = em_trainable_flag))
    model.add(Dropout(dropout))
    model.add(Conv1D(n_filters,kernel_size=3,activation=activation))
    model.add(Bidirectional(LSTM(lstm_out, return_sequences=True,recurrent_dropout=dropout)))
    model.add(Attention())
    
    for i in range(num_hidden_layers):
        model.add(Dense(num_dense_nodes,activation=activation))
        
    model.add(Dense(27,activation='softmax'))
        
    optimizer = Adam(lr=learning_rate)
    
    model.compile(loss = 'categorical_crossentropy', optimizer=optimizer,metrics = ['accuracy'])
    
    return model

path_best_model = 'cnn_lstm_attention.keras'


record = dict()
key=0

def get_class_from_pred(pred):
    return [label_encoding[x.argmax()] for x in pred]

@use_named_args(dimensions=parameters_2)
def fitness(learning_rate,num_hidden_layers,num_dense_nodes,activation,dropout,batch_size,epoch,lstm_out,filter_size,n_filters):
    global key
    global record
    
    # Print the hyper-parameters.
    print('-----------------------------combination no={0}------------------'.format(key))
    print('learning rate===> '.format(learning_rate))
    print('num_hidden_layers:', num_hidden_layers)
    print('num_dense_nodes:', num_dense_nodes)
    print('activation:', activation)
    print('lstm_out:', lstm_out)
    print('dropout===>'.format(dropout))
    print('batch_size===> ',batch_size)
    print('epoch===> ',epoch)
    print('filter_size===> ',filter_size)
    print('num_filters===> 'n_filters)

    print()
    
    
    acc=[]
    f1_Score=[]
    for train,test in kfold.split(dataX,aspect):
        model = define_model_2(learning_rate=learning_rate,
                             num_hidden_layers=num_hidden_layers,
                             num_dense_nodes=num_dense_nodes,
                             activation=activation,
                             lstm_out=lstm_out,
                             dropout=dropout,
                             em='embedding_matrix',
                             em_trainable_flag=False,
                             em_dim=300,
                             n_filters=n_filters,
                             kernel_size=filter_size)

  
   
        history = model.fit(x=dataX,
                            y=aspect,
                            epochs=epoch,
                            batch_size=batch_size,
                            )

        pred1 = model.predict(test_headline_X)
        pred_class = get_class_from_pred(pred1)
        f1=f1_score(h_aspect,pred_class,average='macro')
        
        accuracy = history.history['val_acc'][-1]

        # Print the classification accuracy.
        print()
        print("Accuracy: {0:.2%}".format(accuracy))
        print("F1 Score: ",f1)
        print()
        
        f1_Score.append(f1)
        acc.append(accuracy)
        
        record[key] = {'parameters':[learning_rate,num_hidden_layers,num_dense_nodes,activation,dropout,batch_size,epoch,lstm_out,filter_size,n_filters],'f1_score':f1,'accuracy':accuracy}
        
        with open('record.json', 'w') as fout:
               json.dump(record,fout,indent=4)
    
        key+=1
    
        del model

        K.clear_session()
        return -accuracy

        
        

seed=7
np.random.seed(seed)

kfold = KFold(n_splits=10, shuffle=True, random_state=seed)

fitness(x=default_parameters_2)


search_result = gp_minimize(func=fitness,
                            dimensions=parameters_2,
                            acq_func='EI', # Expected Improvement.
                            n_calls=11,
                            x0=default_parameters_2)

space = search_result.space


space.point_to_dict(search_result.x)


search_result.fun


sorted(zip(search_result.func_vals, search_result.x_iters))


# IAN 

def define_model_3(learning_rate,dropout,lstm_out,em,em_trainable_flag, em_dim,activation,num_dense_nodes):
    
    input_context = Input(shape=(max_length,),name='Context')
    input_target = Input(shape=(max_length,),name='Target')
    
    embedding=Embedding(vocab_size, len(eval(em)[0]), weights = [eval(em)],input_length=max_length,trainable = em_trainable_flag)

    context=embedding(input_context)
    target= embedding(input_target)
    
 
    H_c , _ , _ = LSTM( lstm_out, recurrent_dropout=dropout,return_state=True,return_sequences=True,name="LSTM_C")(context)
    H_t , _ , _ = LSTM( lstm_out, recurrent_dropout=dropout,return_state=True,return_sequences=True,name="LSTM_T")(target)

    
    c_avg = GlobalAveragePooling1D(name='POOL_C')(H_c)  
    t_avg = GlobalAveragePooling1D(name='POOL_T')(H_t)
    

    #c_r = Lambda(lambda x: one_step_attention(x[0],x[1],11))([H_c,t_avg])
    #t_r = Lambda(lambda x: one_step_attention(x[0],x[1],11))([H_t,c_avg])
    
    c_ = RepeatVector(11)(t_avg)
    c_ = concatenator([H_c, c_])
    c_ = densor(c_)
    c_ = activator(c_)
    c = dotor([c_, H_c])    
    c_r= Lambda(lambda x: K.sum(x, axis=1))(c)
    
    t_ = RepeatVector(11)(c_avg)
    t_ = concatenator([H_t, t_])
    t_ = densor(t_)
    t_ = activator(t_)
    t = dotor([t_, H_t])    
    t_r= Lambda(lambda x: K.sum(x, axis=1))(t)

    d = concatenate(inputs=[c_r , t_r])
    
    
    x=Dense(num_dense_nodes,activation=activation)(d)
        
    out=Dense(27,activation='softmax')(x)
   
    IAN_model= Model(inputs=[input_context,input_target],outputs=out)
    
    optimizer = Adam(lr=learning_rate)
    
    IAN_model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
        
    return  IAN_model
   

path_best_model = 'ian_model.keras'


record = dict()
key=0

def get_class_from_pred(pred):
    return [label_encoding[x.argmax()] for x in pred]

@use_named_args(dimensions=parameters)
def fitness(learning_rate,num_hidden_layers,num_dense_nodes,activation,dropout,batch_size,epoch,lstm_out):
    global key
    global record
    
    # Print the hyper-parameters.
    print('-----------------------------combination no={0}------------------'.format(key))
    print('learning rate===> '.format(learning_rate))
    print('num_dense_nodes:', num_dense_nodes)
    print('activation:', activation)
    print('lstm_out:', lstm_out)
    print('dropout===>'.format(dropout))
    print('batch_size===> ',batch_size)
    print('epoch===> ',epoch)

    print()
    
    
    acc=[]
    f1_Score=[]
    for train,test in kfold.split(dataX,aspect):
        model = define_model_3(learning_rate=learning_rate,
                             num_dense_nodes=num_dense_nodes,
                             activation=activation,
                             lstm_out=lstm_out,
                             dropout=dropout,
                             em='embedding_matrix',
                             em_trainable_flag=False,
                             em_dim=300)

  
   
        history = model.fit(x=[dataX,target],
                            y=aspect,
                            epochs=epoch,
                            batch_size=batch_size,
                            )

        pred1 = model.predict([test_headline_X,h_target])
        pred_class = get_class_from_pred(pred1)
        f1=f1_score(h_aspect,pred_class,average='macro')
        
        accuracy = history.history['val_acc'][-1]

        # Print the classification accuracy.
        print()
        print("Accuracy: {0:.2%}".format(accuracy))
        print("F1 Score: ",f1)
        print()
        
        f1_Score.append(f1)
        acc.append(accuracy)
        
        record[key] = {'parameters':[learning_rate,num_dense_nodes,activation,dropout,batch_size,epoch,lstm_out],'f1_score':f1,'accuracy':accuracy}
        
        with open('record.json', 'w') as fout:
               json.dump(record,fout,indent=4)
    
        key+=1
    
        del model

        K.clear_session()
        return -accuracy

        
        

seed=7
np.random.seed(seed)

kfold = KFold(n_splits=10, shuffle=True, random_state=seed)

fitness(x=default_parameters)


search_result = gp_minimize(func=fitness,
                            dimensions=parameters,
                            acq_func='EI', # Expected Improvement.
                            n_calls=11,
                            x0=default_parameters)

space = search_result.space


space.point_to_dict(search_result.x)


search_result.fun


sorted(zip(search_result.func_vals, search_result.x_iters))


# Ian + Bidirectional LSTM

def define_model_4(learning_rate,dropout,lstm_out,em,em_trainable_flag, em_dim, num_dense_nodes,activation):
    
    input_context = Input(shape=(max_length,),name='Context')
    input_target = Input(shape=(max_length,),name='Target')
    
    embedding=Embedding(vocab_size, len(eval(em)[0]), weights = [eval(em)],input_length=max_length,trainable = em_trainable_flag)

    context=embedding(input_context)
    target= embedding(input_target)
    
 
    H_c , _ , _,_,_ = Bidirectional(LSTM( lstm_out, recurrent_dropout=dropout,return_state=True,return_sequences=True,name="LSTM_C"))(context)
    H_t , _ , _ ,_,_= Bidirectional(LSTM( lstm_out, recurrent_dropout=dropout,return_state=True,return_sequences=True,name="LSTM_T"))(target)

    
    c_avg = GlobalAveragePooling1D(name='POOL_C')(H_c)  
    t_avg = GlobalAveragePooling1D(name='POOL_T')(H_t)
    

    #c_r = Lambda(lambda x: one_step_attention(x[0],x[1],11))([H_c,t_avg])
    #t_r = Lambda(lambda x: one_step_attention(x[0],x[1],11))([H_t,c_avg])
    
    c_ = RepeatVector(11)(t_avg)
    c_ = concatenator([H_c, c_])
    c_ = densor(c_)
    c_ = activator(c_)
    c = dotor([c_, H_c])    
    c_r= Lambda(lambda x: K.sum(x, axis=1))(c)
    
    t_ = RepeatVector(11)(c_avg)
    t_ = concatenator([H_t, t_])
    t_ = densor(t_)
    t_ = activator(t_)
    t = dotor([t_, H_t])    
    t_r= Lambda(lambda x: K.sum(x, axis=1))(t)

    d = concatenate(inputs=[c_r , t_r])
    
    
    x=Dense(num_dense_nodes,activation=activation)(d)
        
    out=Dense(27,activation='softmax')(x)
   
    IAN_model= Model(inputs=[input_context,input_target],outputs=out)
    
    optimizer = Adam(lr=learning_rate)
    
    IAN_model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
        
    return  IAN_model
   

path_best_model = 'ian_bilistm_model.keras'


record = dict()
key=0

def get_class_from_pred(pred):
    return [label_encoding[x.argmax()] for x in pred]

@use_named_args(dimensions=parameters)
def fitness(learning_rate,num_hidden_layers,num_dense_nodes,activation,dropout,batch_size,epoch,lstm_out):
    global key
    global record
    
    # Print the hyper-parameters.
    print('-----------------------------combination no={0}------------------'.format(key))
    print('learning rate===> '.format(learning_rate))
#    print('num_dense_nodes:', num_dense_nodes)
    print('activation:', activation)
    print('lstm_out:', lstm_out)
    print('dropout===>'.format(dropout))
    print('batch_size===> ',batch_size)
    print('epoch===> ',epoch)

    print()
    
    
    acc=[]
    f1_Score=[]
    for train,test in kfold.split(dataX,aspect):
        model = define_model_4(learning_rate=learning_rate,
                             num_dense_nodes=num_dense_nodes,
                             activation=activation,
                             lstm_out=lstm_out,
                             dropout=dropout,
                             em='embedding_matrix',
                             em_trainable_flag=False,
                             em_dim=300)

  
   
        history = model.fit(x=[dataX,target],
                            y=aspect,
                            epochs=epoch,
                            batch_size=batch_size,
                            )

        pred1 = model.predict([test_headline_X,h_target])
        pred_class = get_class_from_pred(pred1)
        f1=f1_score(h_aspect,pred_class,average='macro')
        
        accuracy = history.history['val_acc'][-1]

        # Print the classification accuracy.
        print()
        print("Accuracy: {0:.2%}".format(accuracy))
        print("F1 Score: ",f1)
        print()
        
        f1_Score.append(f1)
        acc.append(accuracy)
        
        record[key] = {'parameters':[learning_rate,num_dense_nodes,activation,dropout,batch_size,epoch,lstm_out],'f1_score':f1,'accuracy':accuracy}
        
        with open('record.json', 'w') as fout:
               json.dump(record,fout,indent=4)
    
        key+=1
    
        del model

        K.clear_session()
        return -accuracy

        
        

seed=7
np.random.seed(seed)

kfold = KFold(n_splits=10, shuffle=True, random_state=seed)

fitness(x=default_parameters)


search_result = gp_minimize(func=fitness,
                            dimensions=parameters,
                            acq_func='EI', # Expected Improvement.
                            n_calls=11,
                            x0=default_parameters)

space = search_result.space


space.point_to_dict(search_result.x)


search_result.fun


sorted(zip(search_result.func_vals, search_result.x_iters))
